# Cài đặt và import các thư viện cần thiết

In [ ]:
# Install the required libraries
#SAM
!pip install git+https://github.com/facebookresearch/segment-anything.git
#Transformers
!pip install -q git+https://github.com/huggingface/transformers.git
#Datasets to prepare data and monai if you want to use special loss functions
!pip install datasets
!pip install -q monai
#Patchify to divide large images into smaller patches for training. (Not necessary for smaller images)
!pip install patchify

In [ ]:
# Set-up SSIM Loss
!pip install pytorch-msssim

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import os
from patchify import patchify  #Only to handle large images
import random
from scipy import ndimage
import cv2
from datasets import Dataset
from PIL import Image
import matplotlib.pyplot as plt
import random
import torch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Chia dataset để training model

## Chia dataset cho tập train

In [ ]:
# Đường dẫn tới thư mục chứa các hình ảnh
image_folder = "/kaggle/input/otu-2d/OTU_2D/train/Image"

# Lấy danh sách các tệp trong thư mục và sắp xếp theo thứ tự tên tăng dần từ A đến Z
image_files = sorted(os.listdir(image_folder))

# Khởi tạo một danh sách để chứa các hình ảnh dưới dạng mảng NumPy
image_array_list = []

# Lặp qua tất cả các tệp trong thư mục đã sắp xếp
for filename in image_files:
    # Kiểm tra xem tệp có phải là hình ảnh không
    if filename.endswith(('.JPG')):
        # Đường dẫn đầy đủ tới hình ảnh
        img_path = os.path.join(image_folder, filename)

        # Đọc hình ảnh
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (256, 256))

        # Kiểm tra xem hình ảnh có được đọc thành công hay không
        if img is not None:
            # Thêm hình ảnh vào danh sách
            image_array_list.append(img)
        else:
            print(f"Không thể đọc hình ảnh {filename}")

# Chuyển đổi danh sách hình ảnh thành mảng NumPy
train_images_np = np.array(image_array_list)

# In ra kích thước của mảng hình ảnh
print("Kích thước của mảng hình ảnh:", train_images_np.shape)


In [ ]:
# Đường dẫn tới thư mục chứa các hình ảnh
image_folder = "/kaggle/input/otu-2d/OTU_2D/train/Label"

# Lấy danh sách các tệp trong thư mục và sắp xếp theo thứ tự tên tăng dần từ A đến Z
image_files = sorted(os.listdir(image_folder))

# Khởi tạo một danh sách để chứa các hình ảnh dưới dạng mảng NumPy
image_array_list = []

# Lặp qua tất cả các tệp trong thư mục đã được sắp xếp
for filename in image_files:
    # Kiểm tra xem tệp có phải là hình ảnh không
    if filename.endswith('.PNG'):
        # Đường dẫn đầy đủ tới hình ảnh
        img_path = os.path.join(image_folder, filename)

        # Đọc hình ảnh
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 256))
        img = img / 255.0
        (thresh, img) = cv2.threshold(img, 0, 1, cv2.THRESH_BINARY)

        # img = np.int32(img)

        # Kiểm tra xem hình ảnh có được đọc thành công hay không
        if img is not None:
            # Thêm hình ảnh vào danh sách
            image_array_list.append(img)
        else:
            print(f"Không thể đọc hình ảnh {filename}")

# Chuyển đổi danh sách hình ảnh thành mảng NumPy
train_labels_np = np.array(image_array_list)

# In ra kích thước của mảng hình ảnh
print("Kích thước của mảng mặt nạ:", train_labels_np.shape)

In [ ]:
# Convert the NumPy arrays to Pillow images and store them in a dictionary
dataset_dict = {
    "image": [Image.fromarray(img).convert('RGB') for img in train_images_np],
    "label": [Image.fromarray(mask).convert('I') for mask in train_labels_np],
}

# Create the dataset using the datasets.Dataset class
train_dataset = Dataset.from_dict(dataset_dict)

In [ ]:
train_dataset

## Chia dataset cho tập validation

In [ ]:
# Đường dẫn tới thư mục chứa các hình ảnh
image_folder = "/kaggle/input/otu-2d/OTU_2D/validation/Image"

# Lấy danh sách các tệp trong thư mục và sắp xếp theo thứ tự tên tăng dần từ A đến Z
image_files = sorted(os.listdir(image_folder))

# Khởi tạo một danh sách để chứa các hình ảnh dưới dạng mảng NumPy
image_array_list = []

# Lặp qua tất cả các tệp trong thư mục đã sắp xếp
for filename in image_files:
    # Kiểm tra xem tệp có phải là hình ảnh không
    if filename.endswith(('.JPG')):
        # Đường dẫn đầy đủ tới hình ảnh
        img_path = os.path.join(image_folder, filename)

        # Đọc hình ảnh
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (256, 256))

        # Kiểm tra xem hình ảnh có được đọc thành công hay không
        if img is not None:
            # Thêm hình ảnh vào danh sách
            image_array_list.append(img)
        else:
            print(f"Không thể đọc hình ảnh {filename}")

# Chuyển đổi danh sách hình ảnh thành mảng NumPy
val_images_np = np.array(image_array_list)

# In ra kích thước của mảng hình ảnh
print("Kích thước của mảng hình ảnh:", val_images_np.shape)


In [ ]:
# Đường dẫn tới thư mục chứa các hình ảnh
image_folder = "/kaggle/input/otu-2d/OTU_2D/validation/Label"

# Lấy danh sách các tệp trong thư mục và sắp xếp theo thứ tự tên tăng dần từ A đến Z
image_files = sorted(os.listdir(image_folder))

# Khởi tạo một danh sách để chứa các hình ảnh dưới dạng mảng NumPy
image_array_list = []

# Lặp qua tất cả các tệp trong thư mục đã được sắp xếp
for filename in image_files:
    # Kiểm tra xem tệp có phải là hình ảnh không
    if filename.endswith('.PNG'):
        # Đường dẫn đầy đủ tới hình ảnh
        img_path = os.path.join(image_folder, filename)

        # Đọc hình ảnh
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 256))
        img = img / 255.0
        (thresh, img) = cv2.threshold(img, 0, 1, cv2.THRESH_BINARY)

        # img = np.int32(img)

        # Kiểm tra xem hình ảnh có được đọc thành công hay không
        if img is not None:
            # Thêm hình ảnh vào danh sách
            image_array_list.append(img)
        else:
            print(f"Không thể đọc hình ảnh {filename}")

# Chuyển đổi danh sách hình ảnh thành mảng NumPy
val_labels_np = np.array(image_array_list)

# In ra kích thước của mảng hình ảnh
print("Kích thước của mảng mặt nạ:", val_labels_np.shape)

In [ ]:
# Convert the NumPy arrays to Pillow images and store them in a dictionary
dataset_dict = {
    "image": [Image.fromarray(img).convert('RGB') for img in val_images_np],
    "label": [Image.fromarray(mask).convert('I') for mask in val_labels_np],
}

# Create the dataset using the datasets.Dataset class
val_dataset = Dataset.from_dict(dataset_dict)

In [ ]:
val_dataset

## Chia dataset cho tập test

In [ ]:
# # Đường dẫn tới thư mục chứa các hình ảnh
# image_folder = "/kaggle/input/otu-2d/OTU_2D/test/Image"

# # Lấy danh sách các tệp trong thư mục và sắp xếp theo thứ tự tên tăng dần từ A đến Z
# image_files = sorted(os.listdir(image_folder))

# # Khởi tạo một danh sách để chứa các hình ảnh dưới dạng mảng NumPy
# image_array_list = []

# # Lặp qua tất cả các tệp trong thư mục đã sắp xếp
# for filename in image_files:
#     # Kiểm tra xem tệp có phải là hình ảnh không
#     if filename.endswith(('.JPG')):
#         # Đường dẫn đầy đủ tới hình ảnh
#         img_path = os.path.join(image_folder, filename)

#         # Đọc hình ảnh
#         img = cv2.imread(img_path, cv2.IMREAD_COLOR)
#         img = cv2.resize(img, (256, 256))

#         # Kiểm tra xem hình ảnh có được đọc thành công hay không
#         if img is not None:
#             # Thêm hình ảnh vào danh sách
#             image_array_list.append(img)
#         else:
#             print(f"Không thể đọc hình ảnh {filename}")

# # Chuyển đổi danh sách hình ảnh thành mảng NumPy
# test_images_np = np.array(image_array_list)

# # In ra kích thước của mảng hình ảnh
# print("Kích thước của mảng hình ảnh:", test_images_np.shape)


In [ ]:
# # Đường dẫn tới thư mục chứa các hình ảnh
# image_folder = "/kaggle/input/otu-2d/OTU_2D/test/Label"

# # Lấy danh sách các tệp trong thư mục và sắp xếp theo thứ tự tên tăng dần từ A đến Z
# image_files = sorted(os.listdir(image_folder))

# # Khởi tạo một danh sách để chứa các hình ảnh dưới dạng mảng NumPy
# image_array_list = []

# # Lặp qua tất cả các tệp trong thư mục đã được sắp xếp
# for filename in image_files:
#     # Kiểm tra xem tệp có phải là hình ảnh không
#     if filename.endswith('.PNG'):
#         # Đường dẫn đầy đủ tới hình ảnh
#         img_path = os.path.join(image_folder, filename)

#         # Đọc hình ảnh
#         img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#         img = cv2.resize(img, (256, 256))
#         img = img / 255.0
#         (thresh, img) = cv2.threshold(img, 0, 1, cv2.THRESH_BINARY)

#         # img = np.int32(img)

#         # Kiểm tra xem hình ảnh có được đọc thành công hay không
#         if img is not None:
#             # Thêm hình ảnh vào danh sách
#             image_array_list.append(img)
#         else:
#             print(f"Không thể đọc hình ảnh {filename}")

# # Chuyển đổi danh sách hình ảnh thành mảng NumPy
# test_labels_np = np.array(image_array_list)

# # In ra kích thước của mảng hình ảnh
# print("Kích thước của mảng mặt nạ:", test_labels_np.shape)

In [ ]:
# # Convert the NumPy arrays to Pillow images and store them in a dictionary
# dataset_dict = {
#     "image": [Image.fromarray(img).convert('RGB') for img in test_images_np],
#     "label": [Image.fromarray(mask).convert('I') for mask in test_labels_np],
# }

# # Create the dataset using the datasets.Dataset class
# test_dataset = Dataset.from_dict(dataset_dict)

In [ ]:
# test_dataset

# Kiểm tra ảnh và mặt nạ

In [ ]:
img_num = random.randint(0, train_images_np.shape[0]-1)
example = train_dataset[img_num]
image = example["image"]
image

In [ ]:
example_image = train_dataset[img_num]["image"]
example_mask = train_dataset[img_num]["label"]

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Plot the first image on the left
axes[0].imshow(np.array(example_image), cmap='gray')  # Assuming the first image is grayscale
axes[0].set_title("Image")

# Plot the second image on the right
axes[1].imshow(example_mask, cmap='gray')  # Assuming the second image is grayscale
axes[1].set_title("Mask")

# Hide axis ticks and labels
for ax in axes:
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])

# Display the images side by side
plt.show()

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.2])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

fig, axes = plt.subplots()

axes.imshow(np.array(image))
ground_truth_seg = np.array(example["label"])
show_mask(ground_truth_seg, axes)
axes.title.set_text(f"Ground truth mask")
axes.axis("off")

# Vẽ bounding boxes cho mặt nạ

In [ ]:
#Get bounding boxes from mask.
def get_bounding_box(ground_truth_map):
  # get bounding box from mask
  y_indices, x_indices = np.where(ground_truth_map > 0)
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = ground_truth_map.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  bbox = [x_min, y_min, x_max, y_max]

  return bbox

# Training Model

## Hàm tạo 1 dataset input images and mask

In [ ]:
from torch.utils.data import Dataset

class SAMDataset(Dataset):
  """
  This class is used to create a dataset that serves input images and masks.
  It takes a dataset and a processor as input and overrides the __len__ and __getitem__ methods of the Dataset class.
  """
  def __init__(self, dataset, processor):
    self.dataset = dataset
    self.processor = processor

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    item = self.dataset[idx]
    image = item["image"]
    ground_truth_mask = np.array(item["label"])

    # get bounding box prompt | vẽ box cho mặt nạ
    prompt = get_bounding_box(ground_truth_mask)

    # prepare image and prompt for the model | Chuẩn bị mặt nạ và hộp giới hạn
    inputs = self.processor(image, input_boxes=[[prompt]], return_tensors="pt")

    # remove batch dimension which the processor adds by default | Loại bỏ chiều Batch được thêm vào mặc định
    inputs = {k:v.squeeze(0) for k,v in inputs.items()}

    # add ground truth segmentation | Thêm ground truth để đánh giá việc Segment sau này, đánh giá hiệu suất mô hình
    inputs["ground_truth_mask"] = ground_truth_mask

    return inputs

## Load model SAM

### Xử lý dữ liệu để tương thích với đầu vào Model

In [ ]:
# Initialize the processor
from transformers import SamProcessor
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

In [ ]:
# Create an instance of the SAMDataset
train_dataset = SAMDataset(dataset=train_dataset, processor=processor)
val_dataset = SAMDataset(dataset=val_dataset, processor=processor)
# test_dataset = SAMDataset(dataset=test_dataset, processor=processor)

In [ ]:
example = train_dataset[0]
for k,v in example.items():
  print(f'{k}: {v.shape}')

In [ ]:
example = val_dataset[0]
for k,v in example.items():
  print(f'{k}: {v.shape}')

In [ ]:
# example = test_dataset[0]
# for k,v in example.items():
#   print(f'{k}: {v.shape}')

In [ ]:
# Create a DataLoader instance for the training dataset
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, drop_last=False)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=True, drop_last=False)
# test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=True, drop_last=False)

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)

In [ ]:
batch = next(iter(val_dataloader))
for k,v in batch.items():
  print(k,v.shape)

In [ ]:
# batch = next(iter(test_dataloader))
# for k,v in batch.items():
#   print(k,v.shape)

In [ ]:
batch["ground_truth_mask"].shape

### Load model Pretrained của Segment Anything

In [ ]:
# Load the model
from transformers import SamModel
model = SamModel.from_pretrained("facebook/sam-vit-base")

# make sure we only compute gradients for mask decoder
for name, param in model.named_parameters():
  if name.startswith("vision_encoder") or name.startswith("prompt_encoder"):
    param.requires_grad_(False)

### Khởi tạo model với hàm tối ưu là Adam, hàm loss là DiceCELoss

In [ ]:
import torch.optim as optim
from monai.losses import DiceLoss, DiceCELoss, DiceFocalLoss

# Khởi tạo optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=0)

# Sử dụng DiceCELoss
seg_loss = DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')

# Sử dụng DiceFocalLoss
# seg_loss = DiceFocalLoss(sigmoid=True, gamma=0.25)

# Sử dụng DiceLoss
# seg_loss = DiceLoss(to_onehot_y=True, softmax=True)



## Xây dựng Hàm Joint Loss

### Hàm Focal_Loss

In [ ]:
# class FocalLoss(nn.Module):
#     '''
#     Multi-class Focal loss implementation
#     '''
#     def __init__(self, gamma=2, weight=None):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         self.weight = weight

#     def forward(self, input, target):
#         """
#         input: [N, C]
#         target: [N, ]
#         """
#         logpt = F.log_softmax(input, dim=1)
#         pt = torch.exp(logpt)
#         logpt = (1-pt)**self.gamma * logpt
#         loss = F.nll_loss(logpt, target, self.weight)
#         return loss

### Load hàm loss đánh giá model

In [ ]:
from tqdm import tqdm
from statistics import mean
import torch
from torch.nn.functional import threshold, normalize
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM

smooth=1e-10
gamma = 2
weight = None
# Define functions for calculating evaluation metrics
def dice(predicted, target):
    true_positive = torch.sum(predicted * target)
    false_negative = torch.sum(target) - true_positive
    false_positive = torch.sum(predicted) - true_positive
    return (2. * true_positive + smooth) / (2. *true_positive + false_negative + false_positive + smooth)

def iou(predicted, target):
    intersection = torch.sum(predicted * target)
    union = torch.sum(predicted) + torch.sum(target) - intersection
    return (intersection + smooth) / (union + smooth)

def recall(predicted, target):
    true_positive = torch.sum(predicted * target)
    false_negative = torch.sum(target) - true_positive
    return (true_positive + smooth) / (true_positive + false_negative + smooth)

def precision(predicted, target):
    true_positive = torch.sum(predicted * target)
    false_positive = torch.sum(predicted) - true_positive
    return (true_positive + smooth) / (true_positive + false_positive + smooth)

def ssim(predicted, target):
    ssim_module = SSIM(data_range=255, size_average=True, channel=3)
    ssim_loss = 1 - ssim_module(predicted, target)
    return ssim_loss
    
def focalLoss(predicted, target):
    logpt = F.log_softmax(predicted, dim=1)
    pt = torch.exp(logpt)
    logpt = (1-pt)**gamma * logpt
    loss = F.nll_loss(logpt, target, weight)
    return loss

def jointLoss(predicted, target):
    loss = (iou(predicted, target) + ssim(predicted, target) + focalLoss(predicted, target))/3
    return loss


### Train Model

In [ ]:
# model.load_state_dict(torch.load("/kaggle/input/validation-100epochs/checkpoint_SAM/best_model_weights.pt"))

### Early Stopping

In [ ]:
from tqdm import tqdm
from statistics import mean
import torch
from torch.nn.functional import threshold, normalize
import os

# Tạo thư mục checkpoint
checkpoint_dir = '/kaggle/working/checkpoint_SAM'
os.makedirs(checkpoint_dir, exist_ok=True)

# Biến để theo dõi loss tốt nhất và trọng số của nó
best_loss = float('inf')
best_weights = None

# Add list to record loss of train dataset
train_loss_list = []
train_dice_loss = []
train_iou_loss = []
train_precision_loss = []
train_recall_loss = []

# Add list to record loss of validation dataset
val_loss_list = []
val_dice_loss = []
val_iou_loss = []
val_precision_loss = []
val_recall_loss = []

# Early stopping parameters
patience = 15  # Số lượng epochs mà mô hình không cải thiện trước khi dừng sớm
counter = 0

# Training loop
num_epochs = 50

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()
for epoch in range(num_epochs):
    correct_predictions = 0
    total_predictions = 0
    
    # Train Model on Train Dataset
    epoch_losses = []
    train_dice_scores = []
    train_iou_scores = []
    train_recall_scores = []
    train_precision_scores = []
    
    for batch in tqdm(train_dataloader):
        # forward pass
        outputs = model(pixel_values=batch["pixel_values"].to(device),
                        input_boxes=batch["input_boxes"].to(device),
                        multimask_output=False)

        # compute loss
        predicted_masks = outputs.pred_masks.squeeze(1)
        ground_truth_masks = batch["ground_truth_mask"].float().to(device)
        loss = jointLoss(predicted_masks, ground_truth_masks.unsqueeze(1))

        # backward pass (compute gradients of parameters w.r.t. loss)
        optimizer.zero_grad()
        loss.backward()

        # optimize
        optimizer.step()
        epoch_losses.append(loss.item())
        
        predicted_masks_eval = (outputs.pred_masks.squeeze() > 0.5).float() 
        train_dice_scores.append(dice(predicted_masks_eval, ground_truth_masks))
        train_iou_scores.append(iou(predicted_masks_eval, ground_truth_masks))
        train_recall_scores.append(recall(predicted_masks_eval, ground_truth_masks))
        train_precision_scores.append(precision(predicted_masks_eval, ground_truth_masks))

    # Lưu trọng số của mô hình sau mỗi epoch vào thư mục checkpoint
    if epoch == num_epochs - 1:
        torch.save(model.state_dict(), os.path.join(checkpoint_dir, f'model_epoch_{epoch}.pt'))
    else:
        torch.save(model.state_dict(), os.path.join(checkpoint_dir, f'model_epoch_final.pt'))

    # Tính loss trung bình và accuracy của epoch hiện tại
    epoch_loss_mean = mean(epoch_losses)
    train_loss_list.append(epoch_loss_mean)
    
    train_dice = torch.tensor(train_dice_scores).mean().item()
    train_iou = torch.tensor(train_iou_scores).mean().item()
    train_recall = torch.tensor(train_recall_scores).mean().item()
    train_precision = torch.tensor(train_precision_scores).mean().item()
    
    train_dice_loss.append(train_dice)
    train_iou_loss.append(train_iou)
    train_recall_loss.append(train_recall)
    train_precision_loss.append(train_precision)
    
    #-------------------------------------------------------------------------------------
    # Đánh giá mô hình trên tập validation
    validation_losses = []
    val_dice_scores = []
    val_iou_scores = []
    val_recall_scores = []
    val_precision_scores = []
    
    with torch.no_grad():  # Không tính gradient trong quá trình đánh giá
        for batch in tqdm(val_dataloader):
            outputs = model(pixel_values=batch["pixel_values"].to(device),
                            input_boxes=batch["input_boxes"].to(device),
                            multimask_output=False)
            predicted_masks = outputs.pred_masks.squeeze(1)
            ground_truth_masks = batch["ground_truth_mask"].float().to(device)
            loss = jointLoss(predicted_masks, ground_truth_masks.unsqueeze(1))
            validation_losses.append(loss.item())
            
            predicted_masks_eval = (outputs.pred_masks.squeeze() > 0.5).float() 
            val_dice_scores.append(dice(predicted_masks_eval, ground_truth_masks))
            val_iou_scores.append(iou(predicted_masks_eval, ground_truth_masks))
            val_recall_scores.append(recall(predicted_masks_eval, ground_truth_masks))
            val_precision_scores.append(precision(predicted_masks_eval, ground_truth_masks))

    # Tính loss trung bình trên tập validation
    validation_loss_mean = mean(validation_losses)
    val_loss_list.append(validation_loss_mean)
    
    val_dice = torch.tensor(val_dice_scores).mean().item()
    val_iou = torch.tensor(val_iou_scores).mean().item()
    val_recall = torch.tensor(val_recall_scores).mean().item()
    val_precision = torch.tensor(val_precision_scores).mean().item()
    
    val_dice_loss.append(val_dice)
    val_iou_loss.append(val_iou)
    val_recall_loss.append(val_recall)
    val_precision_loss.append(val_precision)
    
    #-------------------------------------------------------------------------------------
    # In thông tin về epoch, loss của tập train và validation
    print(f'EPOCH: {epoch}')
    print(f'Train Mean loss: {epoch_loss_mean:.4f}')
    print(f'Train Dice: {train_dice:.4f}')
    print(f'Train IOU: {train_iou:.4f}')
    print(f'Train Recall: {train_recall:.4f}')
    print(f'Train Precision: {train_precision:.4f}')
    print('----------------------')
    print(f'Validation Mean loss: {validation_loss_mean:.4f}')
    print(f'Validation Dice: {val_dice:.4f}')
    print(f'Validation IOU: {val_iou:.4f}')
    print(f'Validation Recall: {val_recall:.4f}')
    print(f'Validation Precision: {val_precision:.4f}')

    # Kiểm tra xem loss của epoch hiện tại có là tốt nhất không
    if validation_loss_mean < best_loss:
    # Nếu là loss tốt nhất, cập nhật biến best_loss và lưu trọng số tốt nhất
        best_loss = validation_loss_mean
        best_weights = model.state_dict()
        torch.save(best_weights, os.path.join(checkpoint_dir, 'best_model_weights.pt'))
        print("Best model weights saved.")
    print('---------------------------------------------')
    
    #-------------------------------------------------------------------------------------
    # Kiểm tra early stopping
    if epoch > 0:  # Bắt đầu kiểm tra early stopping sau epoch đầu tiên
        if validation_loss_mean >= prev_epoch_loss:
            counter += 1
            if counter >= patience:
                print(f"Early stopping! No improvement in {patience} epochs.")
                break
        else:
            counter = 0  # Reset counter
            prev_epoch_loss = validation_loss_mean  # Lưu loss của epoch hiện tại để so sánh với epoch tiếp theo
    else:
        prev_epoch_loss = validation_loss_mean

In [ ]:
train_loss_list

In [ ]:
train_dice_loss

In [ ]:
train_iou_loss

In [ ]:
train_recall_loss

In [ ]:
train_precision_loss

In [ ]:
val_loss_list

In [ ]:
val_dice_loss

In [ ]:
val_iou_loss

In [ ]:
val_recall_loss

In [ ]:
val_precision_loss

# Draw chart for loss between train and validation dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

loss = np.arange(epoch + 1)

# Biểu đồ mean loss
plt.plot(loss, train_loss_list, label='Training Loss')
plt.plot(loss, val_loss_list, label='Validation Loss') 
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

### Draw chart for train dataset

In [ ]:
loss = np.arange(epoch + 1)

# Biểu đồ mean loss
plt.plot(loss, train_dice_loss, label='Dice') 
plt.plot(loss, train_iou_loss, label='IOU') 
plt.plot(loss, train_precision_loss, label='Precision') 
plt.plot(loss, train_recall_loss, label='Recall') 
plt.xlabel('Epoch')
plt.ylabel('')
plt.title('Metrics for training dataset')
plt.legend()
plt.show()


### Draw chart for validation dataset

In [ ]:
loss = np.arange(epoch + 1)

# Biểu đồ mean loss
plt.plot(loss, val_dice_loss, label='Dice') 
plt.plot(loss, val_iou_loss, label='IOU') 
plt.plot(loss, val_precision_loss, label='Precision') 
plt.plot(loss, val_recall_loss, label='Recall') 
plt.xlabel('Epoch')
plt.ylabel('')
plt.title('Metrics for validation dataset')
plt.legend()
plt.show()


### Đánh giá trọng số mô hình với tập Test Dataset (IOU, Precision, Recall, Dice)

In [ ]:
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device
model.to(device)

# Tải trọng số từ checkpoint
checkpoint_path = "/kaggle/input/validation-100epochs/checkpoint_SAM/best_model_weights.pt"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

# Load trọng số vào mô hình
model.load_state_dict(checkpoint)

# Đặt mô hình vào chế độ đánh giá
model.eval()

# Tiếp tục quá trình kiểm tra mô hình như đã thực hiện trước đó
val_dice_scores = []
val_iou_scores = []
val_recall_scores = []
val_precision_scores = []

with torch.no_grad():
    for batch in tqdm(val_dataloader):
        # Forward pass
        outputs = model(pixel_values=batch["pixel_values"].to(device),
                        input_boxes=batch["input_boxes"].to(device),
                        multimask_output=False)

        # Compute evaluation metrics
        predicted_masks = (outputs.pred_masks.squeeze() > 0.5).float() 
        ground_truth_masks = batch["ground_truth_mask"].float().to(device)

        val_dice_scores.append(dice(predicted_masks, ground_truth_masks))
        val_iou_scores.append(iou(predicted_masks, ground_truth_masks))
        val_recall_scores.append(recall(predicted_masks, ground_truth_masks))
        val_precision_scores.append(precision(predicted_masks, ground_truth_masks))

# Print evaluation metrics
print("\n")
print(f'Validation Dice: {torch.tensor(val_dice_scores).mean().item():.4f}')
print(f'Validation IOU: {torch.tensor(val_iou_scores).mean().item():.4f}')
print(f'Validation Recall: {torch.tensor(val_recall_scores).mean().item():.4f}')
print(f'Validation Precision: {torch.tensor(val_precision_scores).mean().item():.4f}')


In [ ]:
# # Check if GPU is available, otherwise use CPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Move model to the device
# model.to(device)

# # Tải trọng số từ checkpoint
# checkpoint_path = "/kaggle/input/validation-100epochs/checkpoint_SAM/best_model_weights.pt"
# checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

# # Load trọng số vào mô hình
# model.load_state_dict(checkpoint)

# # Đặt mô hình vào chế độ đánh giá
# model.eval()

# # Tiếp tục quá trình kiểm tra mô hình như đã thực hiện trước đó
# test_dice_scores = []
# test_iou_scores = []
# test_recall_scores = []
# test_precision_scores = []

# with torch.no_grad():
#     for batch in tqdm(test_dataloader):
#         # Forward pass
#         outputs = model(pixel_values=batch["pixel_values"].to(device),
#                         input_boxes=batch["input_boxes"].to(device),
#                         multimask_output=False)

#         # Compute evaluation metrics
#         predicted_masks = (torch.sigmoid(outputs['pred_masks']).squeeze() > 0.5).float()
#         ground_truth_masks = batch["ground_truth_mask"].float().to(device)

#         test_dice_scores.append(dice(predicted_masks, ground_truth_masks))
#         test_iou_scores.append(iou(predicted_masks, ground_truth_masks))
#         test_recall_scores.append(recall(predicted_masks, ground_truth_masks))
#         test_precision_scores.append(precision(predicted_masks, ground_truth_masks))

# # Print evaluation metrics
# print("\n")
# print(f'Test Dice: {torch.tensor(test_dice_scores).mean().item():.4f}')
# print(f'Test IOU: {torch.tensor(test_iou_scores).mean().item():.4f}')
# print(f'Test Recall: {torch.tensor(test_recall_scores).mean().item():.4f}')
# print(f'Test Precision: {torch.tensor(test_precision_scores).mean().item():.4f}')

### Train tiếp model từ trọng số

## Kiểm thử mô hình

In [ ]:
# import numpy as np
# from PIL import Image

# # let's take a random training example
# idx = 100

# # load image
# image = test_dataset[idx]["image"]
# image

In [ ]:
# import torch
# # Sử dụng GPU nếu có sẵn
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # get box prompt based on ground truth segmentation map
# ground_truth_mask = np.array(test_dataset[idx]["label"])
# prompt = get_bounding_box(ground_truth_mask)

# # prepare image + box prompt for the model
# inputs = processor(image, input_boxes=[[prompt]], return_tensors="pt").to(device)
# for k,v in inputs.items():
#   print(k,v.shape)

In [ ]:
# model.eval()

# # forward pass
# with torch.no_grad():
#   outputs = model(**inputs, multimask_output=False)

In [ ]:
# # apply sigmoid
# medsam_seg_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
# # convert soft mask to hard mask
# medsam_seg_prob = medsam_seg_prob.cpu().numpy().squeeze()
# medsam_seg = (medsam_seg_prob > 0.5).astype(np.uint8)

In [ ]:
# def show_mask(mask, ax, random_color=False):
#     if random_color:
#         color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
#     else:
#         color = np.array([30/255, 144/255, 255/255, 0.6])
#     h, w = mask.shape[-2:]
#     mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
#     ax.imshow(mask_image)

In [ ]:
# import numpy as np
# from PIL import Image
# import torch
# import matplotlib.pyplot as plt

# # Sử dụng GPU nếu có sẵn
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# for idx in range(0, len(test_dataset)):  # Duyệt qua các chỉ số từ 1 đến 10
#     # load image
#     image = test_dataset[idx]["image"]

#     # get box prompt based on ground truth segmentation map
#     ground_truth_mask = np.array(test_dataset[idx]["label"])
#     prompt = get_bounding_box(ground_truth_mask)

#     # prepare image + box prompt for the model
#     inputs = processor(image, input_boxes=[[prompt]], return_tensors="pt").to(device)

#     model.eval()

#     # forward pass
#     with torch.no_grad():
#         outputs = model(**inputs, multimask_output=False)

#     # apply sigmoid
#     medsam_seg_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
#     # convert soft mask to hard mask
#     medsam_seg_prob = medsam_seg_prob.cpu().numpy().squeeze()
#     medsam_seg = (medsam_seg_prob > 0.5).astype(np.uint8)
    
#     fig, axes = plt.subplots(1, 3, figsize=(15, 5))
#     # Hiển thị hình ảnh
#     axes[0].imshow(image)
#     axes[0].set_title("Image")
#     axes[0].axis("off")

#     # Hiển thị đồ thị cho ground truth mask
#     axes[1].imshow(ground_truth_mask)
#     axes[1].set_title("Ground truth mask")
#     axes[1].axis("off")

#     # Hiển thị đồ thị cho predicted mask
#     axes[2].imshow(medsam_seg)
#     axes[2].set_title("Predicted mask")
#     axes[2].axis("off")

#     plt.show()